# 식신사이트 크롤링

In [13]:
import pandas as pd
from urllib.parse import quote
import requests
from bs4 import BeautifulSoup

In [25]:
# https://www.siksinhot.com/ko/search?keywords=%EC%8B%A0%EB%85%BC%ED%98%84%EC%97%AD
site = 'https://www.siksinhot.com/ko/'
url = f"{site}search?keywords={quote('신논현역')}"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

## 1. 필요한 데이터 찾기

In [32]:
# 식당 리스트 저장하기
lis = soup.select_one('.localFood_list').find_all('li')
len(lis)

20

In [52]:
# 한 식당 가져오기
li = lis[0]
li

<li data-reactid="72"><figure data-reactid="73"><a data-reactid="74" href="https://www.siksinhot.com/P/101542" target="_blank"><img alt="원강 , 서울특별시 강남구 학동로6길 16" data-reactid="75" height="448" src="https://img.siksinhot.com/place/1463987224153019.png?w=560&amp;h=448&amp;c=Y" width="560"/><img alt="best" data-reactid="76" src="/static2/images/common/ico_best_3.png"/></a></figure><figcaption data-reactid="77"><a class="textBox" data-reactid="78" href="https://www.siksinhot.com/P/101542" target="_blank"><h2 data-reactid="79">원강</h2><span class="score" data-reactid="80">4.1</span></a><p class="cate" data-reactid="81"><a data-reactid="82" href="/search?keywords=신논현/영동시장">신논현/영동시장</a><!-- react-text: 83 --> / <!-- /react-text --><a data-reactid="84" href="/search?keywords=소구이/불고기">소구이/불고기</a></p><p class="counting" data-reactid="85"><img data-reactid="86" src="/static2/images/search/img/icon_view.png"/><span data-reactid="87">30130</span><img data-reactid="88" src="/static2/images/search/img

In [53]:
# 식당명
store = li.select_one('.textBox > h2').text
store

'원강'

In [54]:
# 평점
# float(), int()로 변환하고자 할때는 반드시 예외처리를 해줘야 함
# 점수가 없을 경우가 있기 때문에
try:
    score = float(li.select_one('.score').text) # 평점이 있을 경우
except:
    score = 0.0 # 평점이 없을 경우 0.0으로 지정

In [67]:
# 장소
place = li.select_one('.cate').find_all('a')[0].text
place

'강남역'

In [69]:
# 음식종류
# menu = li.select_one('.cate').text.split('/')[-1].strip()
menu = li.select_one('.cate').find_all('a')[1].text
menu

'냉면/막국수'

## 2. 리스트에 저장

In [73]:
line = []
for li in lis:
    store = li.select_one('.textBox > h2').text # 식당명
    try:
        score = float(li.select_one('.score').text) # 평점
    except:
        score = 0.0
    place = li.select_one('.cate').find_all('a')[0].text.split('/')[0].strip() # 장소
    menu = li.select_one('.cate').find_all('a')[1].text # 음식종류
    line.append({"상호" : store, "평점" : score, "위치" : place, "메뉴" : menu})

## 3. 데이터 프레임으로 변환

In [74]:
df = pd.DataFrame(line)

In [75]:
df

,상호,평점,위치,메뉴
0,원강,4.1,신논현,소구이/불고기
1,스시려 프리미엄,4.7,강남역,일식
2,영동삼미숯불갈비,4.2,신논현,돼지갈비
3,강남교자 본점,4.2,강남역,칼국수/국수
4,리북집,4.1,신논현,족발/보쌈
5,아빠곰수제돈까스,4.5,신논현,경양식/돈가스
6,도치피자 강남점,4.4,강남역,이탈리안
7,청류벽,4.2,강남역,칼국수/국수
8,함지곱창,4.2,신논현,곱창/대창
9,히까리,4.2,신논현,우동/소바


## 4. csv 파일로 변환

In [77]:
df.to_csv('식신맛집(신논현역).csv')